# NASA Battery Dataset Visualisation and Analysis Using Plotly!
In this notebook, we will be analysing and visualising how the battery paramaters such as Battery Impedance, Electrolyte Resistance (Re), and Charge Transfer Resistance (Rct) are changing as the battery cell is aging through charge/discharge cycles.

# Objective
To visualize changes in Battery_impedance, Re, and Rct as the battery ages using Plotly.
# Relevant Plots
1. Scatter Plot for Re and Rct : Separate plots or traces for each battery (battery_id) to compare performance.
2. Line Plot for Re and Rct Over Time
3. Heatmap of Impedance : To observe patterns across all bateries over time
   


In [1]:
import pandas as pd
from datetime import datetime

file_path = "/home/zuck/Nasa Battery Dataset/cleaned_dataset/metadata.csv"
data_frame = pd.read_csv(file_path)

impedance_data = data_frame[data_frame['type'] == 'impedance'][['start_time', 'Re', 'Rct', 'battery_id']]

impedance_data.head()

,start_time,Re,Rct,battery_id
1,[2010. 7. 21. 16. 53. ...,0.05605783343888099,0.20097016584458333,B0047
3,[2010 7 21 20 31 5],0.05319185850921101,0.16473399914864734,B0047
13,[2010. 7. 22. 17. 3. ...,0.05963791501051059,0.21039872263834902,B0047
15,[2010. 7. 22. 20. 40. 25.5],0.05512505361624278,0.1754882075917004,B0047
17,[2010. 7. 23. 11. 35. ...,0.058878485312444453,0.19095687096090014,B0047


In [2]:
#Since the start_time is not in the correct format so reformatting it to correct type
def convert_to_datetime(entry):
    try:
        if isinstance(entry, str):
            entry = entry.strip("[]").replace(",", "")
            parts = [float(item) for item in entry.split()]
            if len(parts) == 6:
                year, month, day, hour, minute = map(int, parts[:5])
                second = int(parts[5])
                return datetime(year, month, day, hour, minute, second)
    except (ValueError, SyntaxError, TypeError):
        return pd.NaT
    return pd.NaT

impedance_data['start_time'] = impedance_data['start_time'].apply(convert_to_datetime)
impedance_data = impedance_data.dropna(subset=['start_time'])
impedance_data = impedance_data.sort_values(by='start_time')
impedance_data.head()

,start_time,Re,Rct,battery_id
5160,2008-04-18 20:55:29,0.04466870036616091,0.06945627304536996,B0005
5776,2008-04-18 20:55:29,0.03816813609946085,0.06158094574229446,B0007
4544,2008-04-18 20:55:29,0.06123359021000344,0.0785415394665875,B0006
4546,2008-04-18 22:39:16,0.06561684946388616,0.0886832084583472,B0006
5162,2008-04-18 22:39:16,0.04668700162486933,0.07627474098530587,B0005


In [7]:
import plotly.graph_objects as go
import plotly.io as pio

def visualize_impedance_patterns(dataset):
    fig = go.Figure()

    for battery in dataset['battery_id'].unique():
        battery_subset = dataset[dataset['battery_id'] == battery]
        
        fig.add_trace(go.Scatter(
            x=battery_subset['start_time'],
            y=battery_subset['Re'],
            mode='lines+markers',
            name=f'Re (Battery {battery})',
            line=dict(width=2),
            marker=dict(size=6)
        ))
        fig.add_trace(go.Scatter(
            x=battery_subset['start_time'],
            y=battery_subset['Rct'],
            mode='lines+markers',
            name=f'Rct (Battery {battery})',
            line=dict(dash='dot', width=2),
            marker=dict(size=6)
        ))
    
    fig.update_layout(
        title="Battery Impedance Trends",
        xaxis_title="Time (Charge/Discharge Cycles)",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(showgrid=True, tickangle=45),
        yaxis=dict(showgrid=True),
        legend_title="Resistance Metrics"
    )

    fig.show()

visualize_impedance_patterns(impedance_data)

In [16]:
import plotly.graph_objects as go

def create_plot_with_dropdown(impedance_data):
    battery_ids = impedance_data['battery_id'].unique()
    
    fig = go.Figure()

    # Add traces for each battery_id but make them initially invisible
    for battery_id in battery_ids:
        current_battery_info = impedance_data[impedance_data['battery_id'] == battery_id]
        fig.add_trace(go.Scatter(
            x=current_battery_info['start_time'], 
            y=current_battery_info['Re'], 
            mode='lines', 
            name=f'Re (Battery {battery_id})',
            line=dict(color='green'),
            visible=False
        ))
        fig.add_trace(go.Scatter(
            x=current_battery_info['start_time'], 
            y=current_battery_info['Rct'], 
            mode='lines', 
            name=f'Rct (Battery {battery_id})',
            line=dict(color='blue'),
            visible=False
        ))

    # Set the first battery's traces as visible by default
    for i in range(2):  # First two traces (Re and Rct for Battery 1)
        fig.data[i].visible = True

    # Create dropdown options to toggle between batteries
    dropdown_options = []
    for i, battery_id in enumerate(battery_ids):
        visible_traces = [False] * len(fig.data)
        visible_traces[2 * i] = True  # Re trace for current battery_id
        visible_traces[2 * i + 1] = True  # Rct trace for current battery_id
        dropdown_options.append(
            dict(
                label=f"Battery ID {battery_id}",
                method="update",
                args=[{"visible": visible_traces},
                      {"title": f"Resistance (Battery ID: {battery_id})"}]
            )
        )

    # Update layout with dropdown
    fig.update_layout(
        updatemenus=[
            dict(
                buttons=dropdown_options,
                direction="down",
                showactive=True,
                x=0.5,
                y=1.2,
                xanchor="center",
                yanchor="top"
            )
        ],
        title=f"Resistance (Battery ID: {battery_ids[0]})",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=45),
        legend_title="Legend",
        template="plotly"
    )

    fig.show()

# Call the function
create_plot_with_dropdown(impedance_data)


In [ ]:
import plotly.express as px

fig = px.scatter_matrix(
    impedance_data,
    dimensions=['Re', 'Rct'],
    color='battery_id',
    title="Scatter Matrix: Re vs. Rct Across Batteries",
    template="plotly",
    height=700
)

fig.update_traces(
    diagonal_visible=False,
    marker=dict(opacity=0.7, size=7)  
)

fig.update_layout(
    title=dict(x=0.5, xanchor='center'),  
    font=dict(size=12),
    coloraxis_colorbar=dict(title="Battery ID")  

fig.show()


In [23]:
import plotly.graph_objects as go

def create_plot_with_dropdown(impedance_data):
    battery_ids = impedance_data['battery_id'].unique()
    
    fig = go.Figure()

    for battery_id in battery_ids:
        battery_data = impedance_data[impedance_data['battery_id'] == battery_id]
        
        fig.add_trace(go.Scatter(
            x=battery_data['start_time'],
            y=battery_data['Re'],
            mode='lines+markers',
            name=f'Re - Battery {battery_id}',
            visible=False
        ))
        fig.add_trace(go.Scatter(
            x=battery_data['start_time'],
            y=battery_data['Rct'],
            mode='lines+markers',
            name=f'Rct - Battery {battery_id}',
            visible=False
        ))

    for i in range(2):
        fig.data[i].visible = True

    dropdown_options = []
    for i, battery_id in enumerate(battery_ids):
        visible_traces = [False] * len(fig.data)
        visible_traces[2 * i] = True
        visible_traces[2 * i + 1] = True
        dropdown_options.append(
            dict(
                label=f"Battery ID {battery_id}",
                method="update",
                args=[{"visible": visible_traces},
                      {"title": f"Resistance Trends for Battery {battery_id}"}]
            )
        )

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=dropdown_options,
                direction="down",
                showactive=True,
                x=0.5,
                y=1.15,
                xanchor="center",
                yanchor="top"
            )
        ],
        title="Battery Resistance Trends (Re & Rct)",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=45),
        legend=dict(
            x=0.01, 
            y=0.99
        ),
        template="plotly"
    )

    fig.show()

create_plot_with_dropdown(impedance_data)


In [25]:
import plotly.express as px
import warnings

def plot_combined_heatmap_with_dropdown():
    warnings.filterwarnings("ignore")

    battery_options = impedance_data['battery_id'].unique()
    initial_battery = battery_options[0]

    def get_filtered_data(battery, parameter):
        data = impedance_data[impedance_data['battery_id'] == battery]
        data['time'] = data['start_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        return data[['time', 'battery_id', parameter]]

    initial_data = get_filtered_data(initial_battery, 'Re')

    fig = px.density_heatmap(
        initial_data,
        x='time',
        y='battery_id',
        z='Re',
        color_continuous_scale='Cividis',
        title=f"Heatmap of Re for Battery {initial_battery}",
        labels={'time': 'Time', 'battery_id': 'Battery ID', 'Re': 'Resistance (Ohms)'},
        template="plotly"
    )

    dropdown_buttons = []

    for battery in battery_options:
        for parameter in ['Re', 'Rct']:
            filtered_data = get_filtered_data(battery, parameter)
            dropdown_buttons.append(
                {
                    "label": f"{battery} ({parameter})",
                    "method": "update",
                    "args": [
                        {"x": [filtered_data['time']],
                         "y": [filtered_data['battery_id']],
                         "z": [filtered_data[parameter]]},
                        {"title": f"Heatmap of {parameter} for Battery {battery}"}
                    ]
                }
            )

    fig.update_layout(
        updatemenus=[
            {
                "buttons": dropdown_buttons,
                "direction": "down",
                "showactive": True,
                "x": 0.5,
                "y": 1.15,
                "xanchor": "center",
                "yanchor": "top",
            }
        ],
        xaxis=dict(tickangle=45),
        xaxis_title="Time",
        yaxis_title="Battery ID"
    )

    fig.show()
    warnings.resetwarnings()

plot_combined_heatmap_with_dropdown()
